In [1]:
libraries = c("dplyr", "magrittr", "tidyr", "surveillance", "rstan") 
for(x in libraries) { library(x,character.only=TRUE,warn.conflicts=FALSE,quietly=TRUE) }

'%&%' = function(x,y)paste0(x,y)

R.Version()$version.string

This is surveillance 1.18.0. For overview type ‘help(surveillance)’.

Registered S3 method overwritten by 'cli':
  method     from    
  print.boxx spatstat

rstan (Version 2.19.3, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)



[1] "R version 4.0.0 (2020-04-24)"

# Loading the dataset

In [2]:
datestar = as.Date("2020-05-10")
datemin = as.Date("2019-12-25") # particular choice

read.csv("../data/JapaneseDataCOVID19 ("%&%format(datestar,"%y%m%d")%&%").csv") %>%
    mutate(report = if_else(is.na(confirmed), reported, confirmed), report = 
           as.Date(report), onset = as.Date(onset)) %>%
    select(-confirmed, -reported) -> df

(tstar = as.numeric(datestar - datemin))

df %<>% mutate(time_onset = as.numeric(onset - datemin),  
               time_report = as.numeric(report - datemin))

df %>% head

[1] 137

,exp_type,onset,is_asymptomatic,report,time_onset,time_report
,<chr>,<date>,<int>,<date>,<dbl>,<dbl>
1,imported,2020-01-03,0,NA,9,NA
2,imported,2020-01-14,0,NA,20,NA
3,imported,2020-01-21,0,NA,27,NA
4,imported,2020-01-23,0,NA,29,NA
5,imported,2020-01-22,0,NA,28,NA
6,imported,2020-01-26,0,NA,32,NA


In [3]:
# Changing it to wide format respectively to time of illness onset
df %>% select(exp_type,time_onset) %>% group_by(exp_type,time_onset) %>% summarize(n=n()) %>%
    spread(exp_type,n) %>% 
    right_join(expand_grid(time_onset = seq(from = 1, to = tstar-1, by = 1)), by='time_onset') %>% 
    replace(is.na(.), 0) %>%
    rename(time = time_onset) -> Df_cases
Df_cases %>% tail

time,domestic,imported
<dbl>,<int>,<int>
131,30,0
132,19,0
133,18,0
134,8,0
135,5,0
136,0,0


In [4]:
# the same for cases only with observed report date
df %>% filter(is.na(time_onset)) %>% select(exp_type,time_report) %>% group_by(exp_type,time_report) %>% 
    summarize(n=n()) %>% spread(exp_type,n) %>% rename(imported_unobs=imported, domestic_unobs=domestic, time=time_report) %>% 
    right_join(Df_cases, by='time') %>% replace(is.na(.), 0) -> Df_cases
Df_cases %>% tail

time,domestic_unobs,imported_unobs,domestic,imported
<dbl>,<int>,<int>,<int>,<int>
131,33,0,30,0
132,24,0,19,0
133,10,0,18,0
134,17,0,8,0
135,15,0,5,0
136,16,0,0,0


## <font color="purple">Backprojection</font>

In [5]:
# here due to stability of the backprojection algorithm we add zero counts beyond tstar
df_cases = Df_cases %>% rbind(data.frame(time=seq(tstar,tstar+10,1), imported=0, imported_unobs=0, domestic=0, domestic_unobs=0))
df_cases %>% tail

time,domestic_unobs,imported_unobs,domestic,imported
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
142,0,0,0,0
143,0,0,0,0
144,0,0,0,0
145,0,0,0,0
146,0,0,0,0
147,0,0,0,0


In [6]:
K = nrow(df_cases)

# Incubation time
inc_fit = list(meanlog=1.519, sdlog=0.615)
incubation_probability = plnorm(1:K, inc_fit$meanlog, inc_fit$sdlog) - plnorm(1:K-1, inc_fit$meanlog, inc_fit$sdlog)
inc_pmf = c(0,incubation_probability[1:41])
k_used = c(2,2) #parameter of smoothing primary vs secondary

epsilon = 1e-3

## import
if (sum(df_cases$imported)>0) {
    sts = new("sts", epoch=df_cases$time, observed=df_cases$imported)
    bpnp.control = list(k = k_used[1], eps = rep(1e-4,2), iter.max=rep(1000,2), 
                        Tmark = tstar-1,  ##attention
                        B = -1, alpha = 0.01, verbose = FALSE, lambda0 = NULL, 
                        eq3a.method = c("R","C"))
    sts_bp = backprojNP(sts, incu.pmf=inc_pmf, control=modifyList(bpnp.control,list(eq3a.method="C")))
    # output result
    df_cases['imported_backproj'] = upperbound(sts_bp) %>% as.numeric %>% replace(. < epsilon, 0) 
    df_cases['imported_backproj'] = df_cases['imported_backproj']/sum(df_cases['imported_backproj'])*sum(df_cases$imported)
} else 
    df_cases['imported_backproj'] = 0

## secondary
if (sum(df_cases$domestic)>0) {
    sts = new("sts", epoch=df_cases$time, observed=df_cases$domestic)
    bpnp.control = list(k = k_used[2], eps = rep(1e-4,2), iter.max=rep(1000,2), 
                        Tmark = tstar-1, ##attention
                        B = -1, alpha = 0.01, verbose = FALSE, lambda0 = NULL, 
                        eq3a.method = c("R","C"))
    sts_bp = backprojNP(sts, incu.pmf=inc_pmf, control=modifyList(bpnp.control,list(eq3a.method="C")))
    df_cases['domestic_backproj'] = upperbound(sts_bp) %>% as.numeric %>% replace(. < epsilon, 0) 
    df_cases['domestic_backproj'] = df_cases['domestic_backproj']/sum(df_cases['domestic_backproj'])*sum(df_cases$domestic)
} else
    df_cases['domestic_backproj'] = 0
df_cases %<>% ungroup

df_cases %>% tail(30)

time,domestic_unobs,imported_unobs,domestic,imported,imported_backproj,domestic_backproj
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
118,120,0,155,0,0,98.262752729
119,58,0,162,0,0,94.086335109
120,87,0,106,0,0,87.094886897
121,76,0,148,0,0,76.768161336
122,50,0,115,0,0,63.569456097
123,22,0,105,0,0,49.086724506
124,40,0,120,0,0,34.978929279
125,105,0,94,0,0,22.030147212
126,63,0,69,0,0,11.532751350


## <font color="purple">Backprojection including cases with unobserved date of illness onset</font>

In [7]:
# first backprojecting to the suggesting date of illness onset

# Reporting delay: from onset to confirmation
onset2report_fit = list(param1 = 1.741, param2 = 8.573)
onset2report_probability = pweibull(1:K, onset2report_fit$param1, scale=onset2report_fit$param2) - pweibull(1:K-1, onset2report_fit$param1, scale=onset2report_fit$param2)
onset2report_pmf = c(0,onset2report_probability[1:41])

## import
if (sum(df_cases$imported_unobs)>0) {
    sts = new("sts", epoch=df_cases$time, observed=df_cases$imported_unobs)
    bpnp.control = list(k = k_used[1], eps = rep(1e-4,2), iter.max=rep(1000,2), 
                        Tmark = tstar - 1, #attention
                        B = -1, alpha = 0.01, verbose = FALSE, lambda0 = NULL, 
                        eq3a.method = c("R","C"))
    sts_bp = backprojNP(sts, incu.pmf=onset2report_pmf, control=modifyList(bpnp.control,list(eq3a.method="C")))
    # output result
    df_cases['imported_backproj_unobs'] = upperbound(sts_bp) %>% as.numeric %>% replace(. < epsilon, 0) 
    df_cases['imported_backproj_unobs'] = df_cases['imported_backproj_unobs']/sum(df_cases['imported_backproj_unobs'])*sum(df_cases$imported_unobs)
} else 
    df_cases['imported_backproj_unobs'] = 0

## secondary
if (sum(df_cases$domestic_unobs)>0) {
    sts = new("sts", epoch=df_cases$time, observed=df_cases$domestic_unobs)
    bpnp.control = list(k = k_used[2], eps = rep(1e-3,2), iter.max=rep(1000,2), 
                        Tmark = tstar - 1, #attention
                        B = -1, alpha = 0.01, verbose = FALSE, lambda0 = NULL, 
                        eq3a.method = c("C"))
    sts_bp = backprojNP(sts, incu.pmf=onset2report_pmf, control=modifyList(bpnp.control,list(eq3a.method="C")))
    df_cases['domestic_backproj_unobs'] = upperbound(sts_bp) %>% as.numeric %>% replace(. < epsilon, 0) 
    df_cases['domestic_backproj_unobs'] = df_cases['domestic_backproj_unobs']/sum(df_cases['domestic_backproj_unobs'])*sum(df_cases$domestic_unobs)
} else
    df_cases['domestic_backproj_unobs'] = 0
df_cases %<>% ungroup

df_cases %>% filter(time<tstar) %>% tail(20)

time,domestic_unobs,imported_unobs,domestic,imported,imported_backproj,domestic_backproj,imported_backproj_unobs,domestic_backproj_unobs
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
117,89,0,219,0,0,1.023578e+02,0,53.495304865
118,120,0,155,0,0,9.826275e+01,0,53.901379721
119,58,0,162,0,0,9.408634e+01,0,54.262973835
120,87,0,106,0,0,8.709489e+01,0,53.196433418
121,76,0,148,0,0,7.676816e+01,0,49.402573132
122,50,0,115,0,0,6.356946e+01,0,41.898455089
123,22,0,105,0,0,4.908672e+01,0,30.966531944
124,40,0,120,0,0,3.497893e+01,0,19.195540189
125,105,0,94,0,0,2.203015e+01,0,9.860084127


In [8]:
# Now backprojecting unobs + obs by the incubation period

## import
if (sum(df_cases$imported_unobs)>0) {
    sts = new("sts", epoch=df_cases$time, observed=df_cases$imported_backproj_unobs + df_cases$imported)
    bpnp.control = list(k = k_used[1], eps = rep(1e-4,2), iter.max=rep(1000,2), Tmark = nrow(df_cases)-10,
                        Tmark = tstar - 1, #attention
                        B = -1, alpha = 0.01, verbose = FALSE, lambda0 = NULL, 
                        eq3a.method = c("R","C"))
    sts_bp = backprojNP(sts, incu.pmf=inc_pmf, control=modifyList(bpnp.control,list(eq3a.method="C")))
    # output result
    df_cases['imported_backproj_incl_unobs'] = upperbound(sts_bp) %>% as.numeric %>% replace(. < epsilon, 0) 
    df_cases['imported_backproj_incl_unobs'] = df_cases['imported_backproj_incl_unobs']/sum(df_cases['imported_backproj_incl_unobs'])*sum(df_cases$imported_backproj_unobs + df_cases$imported)
} else 
    df_cases['imported_backproj_incl_unobs'] = 0

## secondary
if (sum(df_cases$domestic_unobs)>0) {
    sts = new("sts", epoch=df_cases$time, observed=df_cases$domestic_backproj_unobs + df_cases$domestic)
    bpnp.control = list(k = k_used[2], eps = rep(1e-4,2), iter.max=rep(1000,2), 
                        Tmark = tstar - 1, #attention
                        B = -1, alpha = 0.01, verbose = FALSE, lambda0 = NULL, 
                        eq3a.method = c("C"))
    sts_bp = backprojNP(sts, incu.pmf=inc_pmf, control=modifyList(bpnp.control,list(eq3a.method="C")))
    df_cases['domestic_backproj_incl_unobs'] = upperbound(sts_bp) %>% as.numeric %>% replace(. < epsilon, 0) 
    df_cases['domestic_backproj_incl_unobs'] = df_cases['domestic_backproj_incl_unobs']/sum(df_cases['domestic_backproj_incl_unobs'])*sum(df_cases$domestic_backproj_unobs + df_cases$domestic)
} else
    df_cases['domestic_backproj_incl_unobs'] = 0
df_cases %<>% ungroup

df_cases %>% filter(time<tstar) %>% tail(30)

time,domestic_unobs,imported_unobs,domestic,imported,imported_backproj,domestic_backproj,imported_backproj_unobs,domestic_backproj_unobs,imported_backproj_incl_unobs,domestic_backproj_incl_unobs
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
107,147,2,371,2,0.003023249,2.543770e+02,0.022070485,1.124614e+02,0.001725188,3.014198e+02
108,171,0,356,0,0.000000000,2.316336e+02,0.003284909,9.508829e+01,0.000000000,2.790692e+02
109,71,0,283,2,0.000000000,2.062971e+02,0.000000000,8.583038e+01,0.000000000,2.539135e+02
110,69,2,324,0,0.000000000,1.832082e+02,0.000000000,8.007359e+01,0.000000000,2.308689e+02
111,111,0,293,0,0.000000000,1.660957e+02,0.000000000,7.443514e+01,0.000000000,2.144991e+02
112,83,0,280,0,0.000000000,1.541181e+02,0.000000000,6.813026e+01,0.000000000,2.044356e+02
113,115,0,226,0,0.000000000,1.443115e+02,0.000000000,6.185787e+01,0.000000000,1.972394e+02
114,124,0,225,0,0.000000000,1.339834e+02,0.000000000,5.706240e+01,0.000000000,1.888575e+02
115,123,0,216,0,0.000000000,1.218354e+02,0.000000000,5.454180e+01,0.000000000,1.761931e+02


In [9]:
sum(df_cases$domestic) + sum(df_cases$domestic_unobs)

[1] 15051

In [10]:
sum(df_cases$domestic_unobs)

[1] 2843

In [11]:
df_cases %<>% select(-domestic_backproj_unobs,-imported_backproj_unobs) %>% filter(time<tstar)
df_cases %>% write.csv("../results/JapaneseDataCOVID19_with_backproj ("%&%format(datestar,"%y%m%d")%&%").csv", row.names=FALSE)

# <font color="green">[R] Effective reproduction number</font>

## <font color="purple">Stan program</font>

In [12]:
## The code is written by Andrei R. Akhmetzhanov and verified by Kenji Mizumoto
"functions {
    /* discretesized version of lognormal distribution */
    vector plognormal(real mu, real sigma, int K) {
        vector[K] res; 
        for (k in 1:K)
            res[k] = exp(lognormal_lcdf(k | mu, sigma));

        return append_row(res[1], res[2:K]-res[1:(K-1)]);        
    }

    /* discretesized version of Weibull distribution */
    vector pweibull(real kappa, real theta, int K) {
        vector[K] res; 
        for (k in 1:K)
            res[k] = -expm1(-pow(1.0*k/theta, kappa));

        return append_row(res[1], res[2:K]-res[1:(K-1)]);  
    }

    /* calculating the convolutions */
    // X: first function, Yrev: reversed version of the second function
    // K: length of X and Yrev
    // the result is the vector of length K-1, because the first element equal zero is omitted
    vector convolution(vector X, vector Yrev, int K) {
        vector[K-1] res;
        res[1] = X[1]*Yrev[K];
        for (k in 2:K-1) 
            res[k] = dot_product(head(X, k), tail(Yrev, k));

        return res;        
    }

    // Special form of the convolution with adjusting to the delay
    // F: cumulative distribution function of the reporting delay
    vector convolution_with_delay(vector X, vector Yrev, vector F, int K) {
        vector[K-1] res;
        vector[K] Z = X ./ F;

        res[1] = F[2]*Z[1]*Yrev[K];
        for (k in 3:K) 
            res[k-1] = F[k]*dot_product(head(Z, k-1), tail(Yrev, k-1));

        return res;        
    }
}

data {
    int<lower = 1> K; //number of days
    vector<lower = 0>[K] imported_backproj;
    vector<lower = 0>[K] domestic_backproj;
    int<lower = K> upper_bound;

    // serial interval
    real<lower = 0> param1_SI;
    real<lower = 0> param2_SI;

    // reporting delay is given by Weibul distribution
    real<lower = 0> param1_delay;
    real<lower = 0> param2_delay;

    // incubation period
    real mu_inc;
    real<lower = 0> sigma_inc;
}

transformed data {
    vector[K] cases_backproj;

    vector[K-1] conv;
    vector[K-1] conv_delay_adj;

    cases_backproj = imported_backproj + domestic_backproj;
    {
        // serial interval
        vector[K] gt = pweibull(param1_SI, param2_SI, K);
        vector[K] gtrev;
 
        vector[upper_bound] IncPeriod = plognormal(mu_inc, sigma_inc, upper_bound); 
        vector[upper_bound] IncPeriod_inv;
        vector[upper_bound] ReportDelay;
        vector[upper_bound] conv_report_inc;
        
        vector[upper_bound] F_tmp;
        vector[K] F; 
        
        for (k in 1:K) 
            gtrev[k] = gt[K+1-k];
        
        for (k in 1:upper_bound)
            IncPeriod_inv[k] = IncPeriod[upper_bound+1-k];

        // vector[upper_bound] ReportDelay;
        ReportDelay = pweibull(param1_delay, param2_delay, upper_bound);

        // convolution of the reporting delay distribution and incubation period
        // vector[upper_bound] conv_report_inc;
        conv_report_inc = append_row(rep_vector(0,1), convolution(ReportDelay, IncPeriod_inv, upper_bound));
        conv_report_inc = cumulative_sum(conv_report_inc);
        
        ReportDelay = cumulative_sum(ReportDelay);
        for (k in 1:upper_bound) 
            F_tmp[k] = ReportDelay[upper_bound+1-k];
        F = head(F_tmp, K);

        // convolution without adjusting for the delay
        conv = convolution(cases_backproj, gtrev, K);

        // convolution with adjusting for the delay
        conv_delay_adj = convolution_with_delay(cases_backproj, gtrev, F, K);
    }
}

parameters {
    // effective reproduction number without adjustment to the delay in reporting
    vector<lower = 0>[K-1] Rt;

    // effective reproduction number with adjustment to the delay in reporting
    vector<lower = 0>[K-1] Rt_adj;
}

model {
    Rt ~ normal(2.4, 2.0);
    Rt_adj ~ normal(2.4, 2.0);
    
    target += gamma_lpdf(tail(domestic_backproj, K-1) | Rt .* conv + 1e-13, 1.0)
            + gamma_lpdf(tail(domestic_backproj, K-1) | Rt_adj .* conv_delay_adj + 1e-13, 1.0);
}" %>% cat(file="fit_infection.stan", sep="", fill=TRUE)

In [13]:
#model discription
stanmodel = NULL
stanmodel = stan_model(file='fit_infection.stan')

## <font color="purple">Data</font>

In [14]:
stan_data = list(
  K = nrow(df_cases),
  imported_backproj = df_cases$imported_backproj,
  domestic_backproj = df_cases$domestic_backproj,
  upper_bound = tstar,
  ## Serial interval [Nishiura et al 2020 - only certain cases]
  param1_SI = 2.305,
  param2_SI = 5.452,
  ## reported delay
  param1_delay = onset2report_fit$param1,
  param2_delay = onset2report_fit$param2,
  ## incubation period [Linton et al 2020 - with right truncation excl. Wuhan residents]
  mu_inc = inc_fit$meanlog,
  sigma_inc = inc_fit$sdlog
  )

## <font color="purple">MCMC settings</font>

In [15]:
Iter =   10000
Warmup =  2000
Thin =      5
Chains =    2
Seed =   1234

## <font color="purple">Model fit</font>

In [16]:
fit <- sampling(stanmodel, 
                data=stan_data,
                iter=Iter,
                warmup=Warmup,
                thin=Thin, 
                chains = Chains, 
                seed=1234)


SAMPLING FOR MODEL 'fit_infection' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 9.5e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.95 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 10000 [  0%]  (Warmup)
Chain 1: Iteration: 1000 / 10000 [ 10%]  (Warmup)
Chain 1: Iteration: 2000 / 10000 [ 20%]  (Warmup)
Chain 1: Iteration: 2001 / 10000 [ 20%]  (Sampling)
Chain 1: Iteration: 3000 / 10000 [ 30%]  (Sampling)
Chain 1: Iteration: 4000 / 10000 [ 40%]  (Sampling)
Chain 1: Iteration: 5000 / 10000 [ 50%]  (Sampling)
Chain 1: Iteration: 6000 / 10000 [ 60%]  (Sampling)
Chain 1: Iteration: 7000 / 10000 [ 70%]  (Sampling)
Chain 1: Iteration: 8000 / 10000 [ 80%]  (Sampling)
Chain 1: Iteration: 9000 / 10000 [ 90%]  (Sampling)
Chain 1: Iteration: 10000 / 10000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 3.43714 seconds (Warm-up)
Chain 1:                6.58638 seconds (Sampling)
Ch

In [17]:
# Check the result
## Rt is for backproj
## Rt_adj is for backproj_incl_unobs
## The first and the last values should be ignored because no information is available and Rt* is unstable
summary(fit)$summary %T>% write.csv("../results/Results of Stan (script C).csv")

,mean,se_mean,sd,2.5%,25%,50%,75%,97.5%,n_eff,Rhat
Rt[1],3.8180378,0.027494934,1.5984212,1.02445551,2.6147852,3.7507505,4.8902418,7.114743,3379.692,1.0002757
Rt[2],3.8362693,0.027205208,1.6071947,0.95057135,2.6545485,3.7581930,4.9085943,7.138335,3490.060,0.9998686
Rt[3],3.7804952,0.027967267,1.5929088,0.96089635,2.6179224,3.7130478,4.8348421,7.060656,3244.007,0.9996142
Rt[4],3.8377390,0.029012003,1.6543991,0.91109538,2.6344046,3.7607146,4.8973353,7.394050,3251.810,0.9999496
Rt[5],3.8806730,0.028651772,1.6179295,1.00955688,2.7222216,3.7796414,4.9130295,7.273546,3188.719,0.9996148
Rt[6],3.8593584,0.028935133,1.6119838,1.08649949,2.6632355,3.7345734,4.9370936,7.226753,3103.633,1.0002405
Rt[7],3.8114733,0.028700708,1.6164243,0.98704246,2.6392918,3.7078472,4.8879106,7.204880,3171.944,0.9995962
Rt[8],3.7581701,0.027841580,1.5986488,1.04602388,2.5637573,3.6561310,4.8381536,7.088932,3296.995,1.0008426
Rt[9],3.7110596,0.028195733,1.5243092,1.03310158,2.6072832,3.5912512,4.7465914,6.903409,2922.667,1.0009952
Rt[10],3.7579668,0.027973464,1.5879607,0.95788470,2.6476593,3.6219204,4.8137265,7.167282,3222.456,0.9996707


In [18]:
# export result Rt with onset date to dataframe for visualization 
data.frame(summary(fit, pars="Rt")$summary) %>% select(mean, X2.5., X97.5.) %>% 
set_colnames(c("Rt", "lower", "upper")) %>%
mutate(onset=seq(datemin+1, by="days", length.out=nrow(df_cases)-1)) %T>% print() %>%
write.csv('../results/rtstan.csv')

# export result Rt_adj with onset date to dataframe for visualization 
data.frame(summary(fit, pars="Rt_adj")$summary) %>% select(mean, X2.5., X97.5.) %>%
set_colnames(c("Rt_adj", "lower", "upper")) %>%
mutate(onset=seq(datemin+1, by="days", length.out=nrow(df_cases)-1)) %T>% print() %>%
write.csv(., '../results/rtadjstan.csv')

            Rt       lower      upper      onset
1   3.81803781 1.024455515 7.11474333 2019-12-26
2   3.83626929 0.950571354 7.13833465 2019-12-27
3   3.78049517 0.960896349 7.06065649 2019-12-28
4   3.83773903 0.911095382 7.39405036 2019-12-29
5   3.88067300 1.009556882 7.27354553 2019-12-30
6   3.85935838 1.086499491 7.22675265 2019-12-31
7   3.81147332 0.987042455 7.20487970 2020-01-01
8   3.75817009 1.046023876 7.08893175 2020-01-02
9   3.71105957 1.033101581 6.90340892 2020-01-03
10  3.75796681 0.957884704 7.16728187 2020-01-04
11  2.47672437 0.455630055 5.37397396 2020-01-05
12  3.07864444 0.640928868 6.14618423 2020-01-06
13  3.52329718 0.871365007 6.79215718 2020-01-07
14  3.70067329 0.949139496 7.11480052 2020-01-08
15  3.76431468 0.952990540 7.08115924 2020-01-09
16  3.59846729 0.880239726 6.85297527 2020-01-10
17  3.14927824 0.746155687 6.24977763 2020-01-11
18  2.47172871 0.453516449 5.28069049 2020-01-12
19  2.50576838 0.500494211 5.12247916 2020-01-13
20  2.98710085 0.691